In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Semnan,IR,2022-04-24 23:16:25,35.5729,53.3971,71.47,26,99,8.08
1,1,Avarua,CK,2022-04-24 23:16:25,-21.2078,-159.7750,80.65,83,40,11.50
2,2,Pisco,PE,2022-04-24 23:15:21,-13.7000,-76.2167,69.85,73,0,16.11
3,3,Coolum Beach,AU,2022-04-24 23:16:26,-26.5333,153.1000,72.10,86,84,16.04
4,4,Mar Del Plata,AR,2022-04-24 23:16:26,-38.0023,-57.5575,66.22,79,0,15.01


In [3]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     539 non-null    int64  
 1   City        539 non-null    object 
 2   Country     532 non-null    object 
 3   Date        539 non-null    object 
 4   Lat         539 non-null    float64
 5   Lng         539 non-null    float64
 6   Max Temp    539 non-null    float64
 7   Humidity    539 non-null    int64  
 8   Cloudiness  539 non-null    int64  
 9   Wind Speed  539 non-null    float64
dtypes: float64(4), int64(3), object(3)
memory usage: 42.2+ KB


In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get only the positive maximum temperature values and convert negative to zero.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [9]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of Wind Speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Avarua,CK,2022-04-24 23:16:25,-21.2078,-159.7750,80.65,83,40,11.50
11,11,Atuona,PF,2022-04-24 23:16:28,-9.8000,-139.0333,79.39,74,15,17.69
19,19,Batticaloa,LK,2022-04-24 23:16:30,7.7102,81.6924,79.81,83,52,5.32
20,20,Nishihara,JP,2022-04-24 23:16:31,26.1842,127.7558,78.46,97,75,8.05
30,30,Jamestown,US,2022-04-24 23:15:42,42.0970,-79.2353,78.44,31,0,10.36
36,36,Aswan,EG,2022-04-24 23:14:51,24.0934,32.9070,79.07,12,6,9.46
37,37,Labuhan,ID,2022-04-24 23:16:36,-6.8844,112.2051,78.24,87,86,6.64
38,38,Kahului,US,2022-04-24 23:16:36,20.8947,-156.4700,84.22,62,20,26.46
40,40,Butaritari,KI,2022-04-24 23:16:37,3.0707,172.7902,81.34,76,100,10.42
45,45,Chake Chake,TZ,2022-04-24 23:17:39,-5.2459,39.7666,77.22,89,96,3.80


In [15]:
preferred_cities_df.count()

City_ID       176
City          176
Country       176
Date          176
Lat           176
Lng           176
Max Temp      176
Humidity      176
Cloudiness    176
Wind Speed    176
dtype: int64

In [16]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Avarua,CK,80.65,-21.2078,-159.7750,
11,Atuona,PF,79.39,-9.8000,-139.0333,
19,Batticaloa,LK,79.81,7.7102,81.6924,
20,Nishihara,JP,78.46,26.1842,127.7558,
30,Jamestown,US,78.44,42.0970,-79.2353,
36,Aswan,EG,79.07,24.0934,32.9070,
37,Labuhan,ID,78.24,-6.8844,112.2051,
38,Kahului,US,84.22,20.8947,-156.4700,
40,Butaritari,KI,81.34,3.0707,172.7902,
45,Chake Chake,TZ,77.22,-5.2459,39.7666,


In [17]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [21]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Avarua,CK,80.65,-21.2078,-159.7750,Paradise Inn
11,Atuona,PF,79.39,-9.8000,-139.0333,Villa Enata
19,Batticaloa,LK,79.81,7.7102,81.6924,Grand Star Guest House
20,Nishihara,JP,78.46,26.1842,127.7558,Minshuku Agaihama
30,Jamestown,US,78.44,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [23]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [26]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [27]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))